In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import re
import time

In [2]:
import pandas as pd
import numpy as np

In [3]:
opts = webdriver.ChromeOptions()
opts.headless =True

In [4]:
path_link = "C:/Users/Sky/.wdm/drivers/chromedriver/win32/101.0.4951.41/chromedriver.exe"

In [17]:
driver = webdriver.Chrome(executable_path = path_link)
driver.implicitly_wait(10)

<ipython-input-17-143182bab761>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = path_link)


In [18]:
search_url = "https://www.proquest.com/washingtonpost?accountid=29989"
driver.get(search_url)


In [19]:
driver.find_element(By.XPATH, '//*[@id="product-level-nav"]/li[2]').click()

In [8]:
keys = """((federal reserve) OR (fed)) AND
((money supply) OR (open market operations) OR
(quantitative easing) OR (monetary policy) OR
(fed funds rate) OR (overnight lending rate) OR
Bernanke OR Volker OR Greenspan OR Yellen OR Powell OR
(central bank) OR (interest rates) OR (fed chairman) OR
(fed chair) OR (lender of last resort) OR (discount window) OR
(European Central Bank) OR ECB OR (Bank of England) OR
(Bank of Japan) OR BOJ OR (Bank of China) OR Bundesbank OR
(Bank of France) OR (Bank of Italy))"""

In [9]:
driver.find_element(By.XPATH,'//*[@id="select_multiDateRange"]/option[10]').click()
# Start
driver.find_element(By.XPATH,'//*[@id="month2"]/option[2]').click()
driver.find_element(By.XPATH,'//*[@id="day2"]/option[2]').click()
driver.find_element(By.XPATH,'//*[@id="year2"]').send_keys('2000')
# End
driver.find_element(By.XPATH,'//*[@id="month2_0"]/option[13]').click()
driver.find_element(By.XPATH,'//*[@id="day2_0"]/option[32]').click()
driver.find_element(By.XPATH,'//*[@id="year2_0"]').send_keys('2021')

In [10]:
driver.find_element(By.XPATH,'//*[@id="SourceType_Newspapers"]').click()
driver.find_element(By.XPATH,'//*[@id="Language_ENG"]').click()

In [11]:
driver.find_element(By.XPATH, '//*[@id="queryTermField"]').send_keys(keys)

In [12]:
#driver.find_element(By.XPATH, '//*[@id="itemsPerPage"]/option[4]').click()

In [13]:
#BASIC_WAIT_LIMIT = 30
#LONG_WAIT_LIMIT = 40
#CHECKBOX_WAIT_LIMIT = 30
#HUMAN_WAIT_LIMIT = 3
#SLEEP_SECONDS = 5

In [14]:
dataset = []
pattern_date = '[0-9]{1,2}\s[A-Za-z]{3,5}\s[0-9]{4}'

while True:
    articles = driver.find_elements(By.CLASS_NAME, 'resultItem')

    for article in articles:
        title = article.find_element(By.XPATH, './/*[@id="citationDocTitleLink"]').get_attribute("title")
        date = article.find_element(By.CLASS_NAME,'newspaperArticle').text
        date = re.search(pattern_date, date).group()
    
        link = article.find_element(By.XPATH, './/*[@id="citationDocTitleLink"]').get_attribute("href")
    
        driver_temp = webdriver.Chrome(executable_path = path_link, options = opts)
        driver_temp.implicitly_wait(10)
        driver_temp.get(str(link))
    
        try:
            full_text = driver_temp.find_element(By.XPATH, '//*[@id="fullTextZone"]/text').text
        except NoSuchElementException:
            full_text = np.nan

        try:
            abstract = driver_temp.find_element(By.CLASS_NAME,'abstract').text
        except NoSuchElementException:
            abstract = np.nan
        
    
        article_info = {'date': date, 'title': title, "abstract": abstract, 'full_text': full_text}
        dataset.append(article_info)
    
        driver_temp.close()
        
    try:
        driver.find_element(By.CSS_SELECTOR, "[title^='Next Page']").click()
        time.sleep(5)
    except NoSuchElementException:
        break

driver.quit()
df = pd.DataFrame(dataset)

<ipython-input-14-37e1a6e9fcaa>:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_temp = webdriver.Chrome(executable_path = path_link, options = opts)


df

In [15]:
df

,date,title,abstract,full_text
0,08 Sep 2006,Fed Official Defends Interest Rate Policy; Yel...,"Janet L. Yellen, president of the Federal Rese...",The Federal Reserve may be done raising intere...
1,07 June 2006,Ben Bernanke: Tough Guy; Fed Chairman Didn't I...,When [Ben S. Bernanke] told Congress in April ...,Ben S. Bernanke rattled world financial market...
2,15 Aug 2019,"In upside-down bond yields, an economic red flag",NaN,For Americans accustomed to paying 4 or 5 perc...
3,05 May 2004,Fed Stays Put on Rates; Central Bank Signals T...,"Long-term interest rates, including for mortga...",Federal Reserve policymakers left interest rat...
4,01 Feb 2006,Fed Raises Rate as Its Chairman Bows Out; Gree...,[Alan Greenspan] received praise and gifts fro...,"Federal Reserve officials, after greeting Alan..."
5,12 Nov 2005,Bernanke Would Lead With Words As Well as Rate...,"[Ben S. Bernanke], in a speech, reassured the ...",Wall Street was growing worried in November 20...
6,21 Mar 2006,"Further Rate Hikes by Fed Uncertain, Bernanke ...","[Ben S. Bernanke], in a speech delivered in Ne...",PUBLISHED CORRECTIONS: A March 21 Business hea...
7,08 Mar 2019,"In Frankfurt, foreshadowing a global slowdown",NaN,The European Central Bank on Thursday unveiled...
8,25 Aug 2013,Central banks' big question: Now what?,Since cutting its main interest rate to near z...,"JACKSON HOLE, WYO. -\nJACKSON HOLE, Wyo. - Eac..."
9,07 Aug 2019,"Fed must be free of political pressure, ex-cha...",NaN,Four former leaders respond to Trump's critici...


In [16]:
#df.to_csv("news_data.csv", index = False)